## Importación de librerías

In [26]:
# Librerías para cálculos y visualización
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

# Importar tu motor de red neuronal
from src.network import NeuralNetwork
from src.dense import Dense
from src.activations import ReLU, Softmax
from src.losses import CrossEntropyLoss
from src.optimizer import Adam
from src.trainer import Trainer


Importamos librerías necesarias y tu implementación propia de la red.

## Cargar y preparar MNIST 

In [ ]:
# Descargar MNIST desde OpenML
mnist = fetch_openml('mnist_784', version=1, cache=True)
X, y = mnist["data"], mnist["target"].astype(np.int64)

# Separar en train/test (60k/10k)
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

# Normalizar imágenes a [0,1]
X_train = X_train / 255.0
X_test = X_test / 255.0

print("MNIST cargado y normalizado correctamente:")
print("X_train:", X_train.shape, "X_test:", X_test.shape)


Se cargan y normalizan las imágenes.

## Codificar etiquetas a one-hot


In [ ]:
# Codificar etiquetas a one-hot (scikit-learn >=1.2)
encoder = OneHotEncoder(sparse_output=False, categories='auto')
y_train_oh = encoder.fit_transform(y_train.reshape(-1,1))
y_test_oh = encoder.transform(y_test.reshape(-1,1))

print("Etiquetas convertidas a one-hot:")
print("y_train_oh:", y_train_oh.shape, "y_test_oh:", y_test_oh.shape)


Tu Trainer requiere que las etiquetas sean one-hot.

## Crear la red neuronal y el trainer


In [ ]:
# Crear la red neuronal
network = NeuralNetwork([
    Dense(784, 128, initialization="he"),
    ReLU(),
    Dense(128, 10, initialization="xavier"),
    Softmax()
])

# Función de pérdida y optimizador
loss_fn = CrossEntropyLoss()
optimizer = Adam(lr=0.001)

# Crear trainer
trainer = Trainer(network, optimizer, loss_fn)

print("Red inicializada correctamente.")


Se define la arquitectura y se crea el entrenador.

## Entrenamiento durante 30 épocas


In [ ]:
# Entrenar la red
history = trainer.train(
    X_train, y_train_oh,
    X_test, y_test_oh,
    epochs=30,
    batch_size=64
)


Entrenamos 30 épocas con batch size 64.
El history guarda pérdida y precisión para gráficas.

## Graficar pérdida y precisión


In [ ]:
# Graficar pérdida y precisión
plt.figure(figsize=(12,5))

# Pérdida
plt.subplot(1,2,1)
plt.plot(history["train_loss"], label="Pérdida entrenamiento")
plt.plot(history["val_loss"], label="Pérdida validación")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Pérdida durante entrenamiento")
plt.legend()

# Precisión
plt.subplot(1,2,2)
plt.plot(history["val_acc"], label="Precisión validación")
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.title("Precisión durante entrenamiento")
plt.legend()

plt.tight_layout()
plt.show()


Se visualizan las métricas de entrenamiento por época.

## Evaluación final en test set


In [ ]:
# Evaluar la red en el conjunto de prueba
val_pred = network.forward(X_test)
val_loss = loss_fn.forward(val_pred, y_test_oh)
val_acc = np.mean(np.argmax(val_pred, axis=1) == np.argmax(y_test_oh, axis=1))

print(f"Pérdida en test: {val_loss:.4f}, Precisión en test: {val_acc:.4f}")


Obtenemos métricas finales para la memoria.

## Guardar gráfica para la memoria


In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history["train_loss"], label="Pérdida entrenamiento")
plt.plot(history["val_loss"], label="Pérdida validación")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Pérdida durante entrenamiento")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history["val_acc"], label="Precisión validación")
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.title("Precisión durante entrenamiento")
plt.legend()

plt.tight_layout()
plt.savefig("resultados_entrenamiento.png")
print("Gráfica guardada como 'resultados_entrenamiento.png'.")


Guardamos la figura para usar en la memoria del proyecto.